Parking Management Systems

billing.py

In [ ]:
def calculate_charges(vehicle_type, hours_parked):
    rates = {
        'Bike': 10,
        'Car': 20,
        'EV': 15,
        'Heavy Vehicle': 40
    }
    rate_per_hour = rates.get(vehicle_type, 20) 
    if hours_parked <= 2:
        charges = rate_per_hour * 2
    else:
        base_charge = rate_per_hour * 2
        additional_hours = hours_parked - 2
        additional_charge = additional_hours * rate_per_hour
        charges = base_charge + additional_charge
    if charges < rate_per_hour:
        charges = rate_per_hour
    
    return round(charges, 2)

parking_system.py

In [ ]:
from datetime import datetime
from billing import calculate_charges

class ParkingLot:
    
    def __init__(self, total_slots=20):
        self.total_slots = total_slots
        self.slots = {}
        self.total_revenue = 0
        self.total_vehicles_today = 0
        self.transactions = []
        
        for slot_num in range(1, total_slots + 1):
            self.slots[slot_num] = None
    
    def find_available_slot(self):
        for slot_num, vehicle_data in self.slots.items():
            if vehicle_data is None:
                return slot_num
        return None
    
    def park_vehicle(self):
        print("\n--- VEHICLE ENTRY ---")
        
        slot_num = self.find_available_slot()
        
        if slot_num is None:
            print("❌ Sorry! Parking lot is FULL. No slots available.")
            return
        
        vehicle_number = input("Enter Vehicle Number (e.g., MH01AB1234): ").upper().strip()
        
        if not vehicle_number:
            print("❌ Vehicle number cannot be empty!")
            return
        
        for slot, data in self.slots.items():
            if data and data['vehicle_number'] == vehicle_number:
                print(f"❌ This vehicle is already parked in Slot {slot}!")
                return
        
        print("\nVehicle Types:")
        print("1. Bike/Scooter")
        print("2. Car/SUV")
        print("3. Electric Vehicle (EV)")
        print("4. Heavy Vehicle (Truck/Bus)")
        
        vehicle_type_choice = input("Select vehicle type (1-4): ")
        
        vehicle_types = {
            '1': 'Bike',
            '2': 'Car',
            '3': 'EV',
            '4': 'Heavy Vehicle'
        }
        
        vehicle_type = vehicle_types.get(vehicle_type_choice, 'Car')
        
        entry_time = datetime.now()
        
        self.slots[slot_num] = {
            'vehicle_number': vehicle_number,
            'vehicle_type': vehicle_type,
            'entry_time': entry_time
        }
        
        self.total_vehicles_today += 1
        
        print(f"\n✅ Vehicle Parked Successfully!")
        print(f"📍 Slot Number: {slot_num}")
        print(f"🚗 Vehicle: {vehicle_number}")
        print(f"🏷️  Type: {vehicle_type}")
        print(f"🕒 Entry Time: {entry_time.strftime('%I:%M %p, %d-%b-%Y')}")
    
    def exit_vehicle(self):
        print("\n--- VEHICLE EXIT ---")
        
        vehicle_number = input("Enter Vehicle Number to exit: ").upper().strip()
        
        slot_num = None
        vehicle_data = None
        
        for slot, data in self.slots.items():
            if data and data['vehicle_number'] == vehicle_number:
                slot_num = slot
                vehicle_data = data
                break
        
        if slot_num is None or vehicle_data is None:
            print(f"❌ Vehicle {vehicle_number} not found in parking lot!")
            return
        
        exit_time = datetime.now()
        entry_time = vehicle_data['entry_time']
        vehicle_type = vehicle_data['vehicle_type']
        
        duration = exit_time - entry_time
        hours_parked = duration.total_seconds() / 3600
        
        charges = calculate_charges(vehicle_type, hours_parked)
        
        print("\n" + "="*50)
        print("📄 PARKING BILL")
        print("="*50)
        print(f"Vehicle Number: {vehicle_number}")
        print(f"Vehicle Type: {vehicle_type}")
        print(f"Slot Number: {slot_num}")
        print(f"Entry Time: {entry_time.strftime('%I:%M %p, %d-%b-%Y')}")
        print(f"Exit Time: {exit_time.strftime('%I:%M %p, %d-%b-%Y')}")
        print(f"Duration: {hours_parked:.2f} hours")
        print(f"💰 Total Charges: ₹{charges:.2f}")
        print("="*50)
        
        self.total_revenue += charges
        
        self.transactions.append({
            'vehicle_number': vehicle_number,
            'vehicle_type': vehicle_type,
            'slot': slot_num,
            'entry_time': entry_time,
            'exit_time': exit_time,
            'duration_hours': hours_parked,
            'charges': charges
        })
        
        self.slots[slot_num] = None
        
        print(f"\n✅ Slot {slot_num} is now available!")
    
    def show_available_slots(self):
        print("\n--- AVAILABLE SLOTS ---")
        
        available_slots = [slot for slot, data in self.slots.items() if data is None]
        
        if available_slots:
            print(f"✅ Available Slots: {len(available_slots)}/{self.total_slots}")
            print(f"Slot Numbers: {', '.join(map(str, available_slots))}")
        else:
            print("❌ No slots available! Parking is FULL.")
    
    def show_parked_vehicles(self):
        print("\n--- CURRENTLY PARKED VEHICLES ---")
        
        parked_count = 0
        
        for slot_num, data in self.slots.items():
            if data:
                parked_count += 1
                entry_time = data['entry_time']
                duration = datetime.now() - entry_time
                hours = duration.total_seconds() / 3600
                
                print(f"\nSlot {slot_num}:")
                print(f"  🚗 Vehicle: {data['vehicle_number']}")
                print(f"  🏷️  Type: {data['vehicle_type']}")
                print(f"  🕒 Parked since: {entry_time.strftime('%I:%M %p')}")
                print(f"  ⏱️  Duration: {hours:.2f} hours")
        
        if parked_count == 0:
            print("ℹ️  No vehicles currently parked.")
        else:
            print(f"\nTotal Parked: {parked_count}/{self.total_slots} slots occupied")
    
    def show_revenue_report(self):
        print("\n" + "="*50)
        print("📊 DAILY REVENUE REPORT")
        print("="*50)
        print(f"📅 Date: {datetime.now().strftime('%d %B %Y')}")
        print(f"🚗 Total Vehicles Today: {self.total_vehicles_today}")
        print(f"💰 Total Revenue: ₹{self.total_revenue:.2f}")
        print(f"📍 Slots Occupied: {sum(1 for data in self.slots.values() if data)}/{self.total_slots}")
        print(f"📍 Slots Available: {sum(1 for data in self.slots.values() if data is None)}/{self.total_slots}")
        
        if self.transactions:
            print(f"\n📝 Total Completed Transactions: {len(self.transactions)}")
            avg_revenue = self.total_revenue / len(self.transactions)
            print(f"📈 Average Revenue per Vehicle: ₹{avg_revenue:.2f}")
        
        print("="*50)

utils.py

In [ ]:
from datetime import datetime

def format_duration(hours):
    if hours < 1:
        minutes = int(hours * 60)
        return f"{minutes} minutes"
    else:
        return f"{hours:.2f} hours"

def validate_vehicle_number(vehicle_num):
    if len(vehicle_num) < 6:
        return False
    return True

def save_report_to_file(report_data, filename="reports/daily_report.txt"):
    with open(filename, 'w') as f:
        f.write(report_data)
    print(f"✅ Report saved to {filename}")

main.py

In [ ]:
from parking_system import ParkingLot

def display_menu():
    print("\n" + "="*50)
    print("🚗 SMART PARKING MANAGEMENT SYSTEM 🚗")
    print("="*50)
    print("1. Park Vehicle (Entry)")
    print("2. Exit Vehicle")
    print("3. View Available Slots")
    print("4. View All Parked Vehicles")
    print("5. Daily Revenue Report")
    print("6. Exit System")
    print("="*50)

def main():
    parking_lot = ParkingLot(total_slots=20)
    
    while True:
        display_menu()
        
        choice = input("\nEnter your choice (1-6): ")
        
        if choice == '1':
            parking_lot.park_vehicle()
        
        elif choice == '2':
            parking_lot.exit_vehicle()
        
        elif choice == '3':
            parking_lot.show_available_slots()
        
        elif choice == '4':
            parking_lot.show_parked_vehicles()
        
        elif choice == '5':
            parking_lot.show_revenue_report()
        
        elif choice == '6':
            print("\n✅ Thank you for using Smart Parking System!")
            print(f"💰 Total Revenue Today: ₹{parking_lot.total_revenue}")
            break
        
        else:
            print("\n❌ Invalid choice! Please enter 1-6")

if __name__ == "__main__":
    main()